In [1]:
import pandas as pd
from utils import load_filtered_data

In [2]:
city = 'Dublin'
name = 'categories'

# load city data
data = load_filtered_data(city)

In [3]:
# creating the nodes

nodes = data['user'][['user_id', 'name']]
nodes = nodes.rename(columns={'user_id': 'id', 'name': 'people_name'})

nodes

,id,people_name
0,_BcWyKQL16ndpBdggh2kNA,Karen
1,8MKygqfspKqG2-X9ycR-Ow,Hunter
2,LiiUu5TS7Bjjoq9xd8tKEQ,Brian
3,JvljCKbnjcDFrWkrqQUdEQ,Frank
4,gfWTee4CnQF-shBgAsvI9g,Pooh
...,...,...
437,vCrD8e4rYHPhN9sHcPQ4Sg,Philip
438,E8ko3g18liOHvKTLduj7cQ,Sara
439,dto2XaLTUIQ-PW5_OJAkcg,Patti
440,CLzR5PlV25MEzOL-lqp5Ow,Wayne


In [4]:
temp_edges_struct = {'from': [], 'to': [], 'weight': [], 'type': []}

# Splitting categories into individual entries
categories_expanded = data['business']['categories'].str.split(', ').explode().dropna().unique()

for category in categories_expanded:
    businesses_in_category = data['business'][data['business']['categories'].apply(lambda x: category in x)]
    users_who_reviewed_category = data['review'][data['review']['business_id'].isin(businesses_in_category['business_id'])]['user_id'].unique()
    for i in range(len(users_who_reviewed_category)-1):
        for j in range(i+1, len(users_who_reviewed_category)):
            # from i to j
            temp_edges_struct['from'].append(users_who_reviewed_category[i])
            temp_edges_struct['to'].append(users_who_reviewed_category[j])
            temp_edges_struct['weight'].append(1)
            temp_edges_struct['type'].append('reviewed-same-category')
            # from j to i
            temp_edges_struct['from'].append(users_who_reviewed_category[j])
            temp_edges_struct['to'].append(users_who_reviewed_category[i])
            temp_edges_struct['weight'].append(1)
            temp_edges_struct['type'].append('reviewed-same-category')

edges = pd.DataFrame(temp_edges_struct)
edges = edges.drop_duplicates()

edges

,from,to,weight,type
0,paCWvJDzJdFDfPMLAUbZlw,BNKwpze_HDmShffe4WqN4A,1,reviewed-same-category
1,BNKwpze_HDmShffe4WqN4A,paCWvJDzJdFDfPMLAUbZlw,1,reviewed-same-category
2,paCWvJDzJdFDfPMLAUbZlw,EuU4rgWWkIDc-hKd7-XaKw,1,reviewed-same-category
3,EuU4rgWWkIDc-hKd7-XaKw,paCWvJDzJdFDfPMLAUbZlw,1,reviewed-same-category
4,paCWvJDzJdFDfPMLAUbZlw,PzHHr5_J8DQOzo4SN_40yA,1,reviewed-same-category
...,...,...,...,...
174805,nsoCXS85JoPBPax2iLtNYw,dto2XaLTUIQ-PW5_OJAkcg,1,reviewed-same-category
174806,dto2XaLTUIQ-PW5_OJAkcg,SkHrxUJB5zx_2ATwVHpfXg,1,reviewed-same-category
174807,SkHrxUJB5zx_2ATwVHpfXg,dto2XaLTUIQ-PW5_OJAkcg,1,reviewed-same-category
174808,dto2XaLTUIQ-PW5_OJAkcg,ZN_WaJ7dgMICieELGtkoVQ,1,reviewed-same-category


In [5]:
# save data
nodes.to_csv(f'nodes_and_edges/{city}_{name}_nodes.csv', index=False)
edges.to_csv(f'nodes_and_edges/{city}_{name}_edges.csv', index=False)